In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters

{'watershed': 'geum_auto'}

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/geum/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['현도_2015.xlsx', '현도_2016.xlsx', '현도_2017.xlsx', '현도_2018.xlsx', '현도_2019.xlsx', '현도_2020.xlsx'], ['대청호_2015.xlsx', '대청호_2016.xlsx', '대청호_2017.xlsx', '대청호_2018.xlsx', '대청호_2019.xlsx', '대청호_2020.xlsx'], ['장계_2015.xlsx', '장계_2016.xlsx', '장계_2017.xlsx', '장계_2018.xlsx', '장계_2019.xlsx', '장계_2020.xlsx']]
data/geum/자동/현도_2015.xlsx
data/geum/자동/현도_2016.xlsx
data/geum/자동/현도_2017.xlsx
data/geum/자동/현도_2018.xlsx
data/geum/자동/현도_2019.xlsx
data/geum/자동/현도_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-10-31 23:00
data/geum/자동/대청호_2015.xlsx
data/geum/자동/대청호_2016.xlsx
data/geum/자동/대청호_2017.xlsx
data/geum/자동/대청호_2018.xlsx
data/geum/자동/대청호_2019.xlsx
data/geum/자동/대청호_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-10-31 23:00
data/geum/자동/장계_2015.xlsx
data/geum/자동/장계_2016.xlsx
data/geum/자동/장계_2017.xlsx
data/geum/자동/장계_2018.xlsx
data/geum/자동/장계_2019.xlsx
data/geum/자동/

1/1 [==============================] - 0s 20ms/step - gen_loss: 7.0137 - disc_loss: 0.3652 - rmse: 0.3357 - val_loss: 0.2902
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.6999 - disc_loss: 0.3493 - rmse: 0.3049 - val_loss: 0.4323
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.0085 - disc_loss: 0.3646 - rmse: 0.2692 - val_loss: 0.3406
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3124 - disc_loss: 0.3467 - rmse: 0.3132 - val_loss: 0.7601
Epoch 55/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.2968 - disc_loss: 0.3496 - rmse: 0.3209 - val_loss: 0.3855
Epoch 56/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.6497 - disc_loss: 0.3404 - rmse: 0.2952 - val_loss: 0.3467
Epoch 57/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8621 - disc_loss: 0.3532 - rmse: 0.3162 - val_loss: 0.4492
Epoch 58/2000
1/1 [======================

Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 19.6147 - disc_loss: 0.2837 - rmse: 0.6678 - val_loss: 0.2352
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.9386 - disc_loss: 0.2818 - rmse: 0.2523 - val_loss: 0.2707
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1482 - disc_loss: 0.2896 - rmse: 0.2619 - val_loss: 0.2459
Epoch 113/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.2679 - disc_loss: 0.2877 - rmse: 0.2738 - val_loss: 0.2548
Epoch 114/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.2333 - disc_loss: 0.2829 - rmse: 0.5734 - val_loss: 0.3020
Epoch 115/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 15.3030 - disc_loss: 0.2898 - rmse: 0.6231 - val_loss: 0.2831
Epoch 116/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 15.6213 - disc_loss: 0.2841 - rmse: 0.6549 - val_loss: 0.4339
Epoch 117/2000
1/

1/1 [==============================] - 0s 20ms/step - gen_loss: 17.9680 - disc_loss: 0.2596 - rmse: 0.6624 - val_loss: 0.2748
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.0990 - disc_loss: 0.2621 - rmse: 0.5411 - val_loss: 0.2606
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4428 - disc_loss: 0.2684 - rmse: 0.2920 - val_loss: 0.4009
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5536 - disc_loss: 0.2427 - rmse: 0.2471 - val_loss: 0.2499
Epoch 172/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.3542 - disc_loss: 0.2564 - rmse: 0.3181 - val_loss: 0.2938
Epoch 173/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.8833 - disc_loss: 0.2502 - rmse: 0.2550 - val_loss: 0.2646
Epoch 174/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3212 - disc_loss: 0.2654 - rmse: 0.2684 - val_loss: 0.2860
Epoch 175/2000
1/1 [=============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4473 - disc_loss: 0.2488 - rmse: 0.2864 - val_loss: 0.2594
Epoch 227/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 11.6077 - disc_loss: 0.2384 - rmse: 0.6898 - val_loss: 0.3378
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.2319 - disc_loss: 0.2413 - rmse: 0.3536 - val_loss: 0.2422
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6749 - disc_loss: 0.2375 - rmse: 0.2726 - val_loss: 0.2274
Epoch 230/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0117 - disc_loss: 0.2325 - rmse: 0.2416 - val_loss: 0.2336
Epoch 231/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3754 - disc_loss: 0.2372 - rmse: 0.2605 - val_loss: 0.2475
Epoch 232/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 14.7116 - disc_loss: 0.2571 - rmse: 0.5461 - val_loss: 0.4208
Epoch 233/2000
1/1 [=============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 18.7327 - disc_loss: 0.2030 - rmse: 0.7832 - val_loss: 0.2348
Epoch 575/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0592 - disc_loss: 0.2055 - rmse: 0.1968 - val_loss: 0.2270
Epoch 576/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.4232 - disc_loss: 0.1884 - rmse: 0.5664 - val_loss: 0.2662
Epoch 577/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.9132 - disc_loss: 0.1968 - rmse: 0.2396 - val_loss: 0.2740
Epoch 578/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.7037 - disc_loss: 0.2061 - rmse: 0.2620 - val_loss: 0.2073
Epoch 579/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.3691 - disc_loss: 0.2038 - rmse: 0.5808 - val_loss: 0.3875
Epoch 580/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.6551 - disc_loss: 0.1979 - rmse: 0.6213 - val_loss: 0.3423
Epoch 581/2000
1/1 [=============

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4005 - disc_loss: 0.1921 - rmse: 0.1984 - val_loss: 0.2328
Epoch 633/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.0644 - disc_loss: 0.1865 - rmse: 0.2954 - val_loss: 0.2130
Epoch 634/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.9649 - disc_loss: 0.1893 - rmse: 0.1858 - val_loss: 0.2042
Epoch 635/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2106 - disc_loss: 0.2057 - rmse: 0.2475 - val_loss: 0.2228
Epoch 636/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8459 - disc_loss: 0.1928 - rmse: 0.2375 - val_loss: 1.0228
Epoch 637/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5102 - disc_loss: 0.2039 - rmse: 0.2228 - val_loss: 0.2169
Epoch 638/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 16.1715 - disc_loss: 0.1859 - rmse: 0.2301 - val_loss: 0.1980
Epoch 639/2000
1/1 [=============

1/1 [==============================] - 0s 20ms/step - gen_loss: 12.5501 - disc_loss: 0.1894 - rmse: 0.2449 - val_loss: 0.2333
Epoch 691/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.2902 - disc_loss: 0.1888 - rmse: 0.2097 - val_loss: 0.2708
Epoch 692/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7097 - disc_loss: 0.1865 - rmse: 0.2364 - val_loss: 0.2011
Epoch 693/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5200 - disc_loss: 0.1762 - rmse: 0.2613 - val_loss: 0.2354
Epoch 694/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 20.2675 - disc_loss: 0.1913 - rmse: 0.3826 - val_loss: 0.6499
Epoch 695/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7549 - disc_loss: 0.1900 - rmse: 0.2152 - val_loss: 0.8890
Epoch 696/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.0736 - disc_loss: 0.1754 - rmse: 0.2710 - val_loss: 0.2317
Epoch 697/2000
1/1 [============

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2950 - disc_loss: 0.1966 - rmse: 0.2151 - val_loss: 0.2039
Epoch 749/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1633 - disc_loss: 0.1760 - rmse: 0.1902 - val_loss: 0.2034
Epoch 750/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3187 - disc_loss: 0.1737 - rmse: 0.2089 - val_loss: 0.1928
Epoch 751/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2036 - disc_loss: 0.1988 - rmse: 0.2134 - val_loss: 0.1955
Epoch 752/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9951 - disc_loss: 0.1874 - rmse: 0.2398 - val_loss: 0.2173
Epoch 753/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.1557 - disc_loss: 0.1915 - rmse: 0.1984 - val_loss: 0.1968
Epoch 754/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.7687 - disc_loss: 0.1751 - rmse: 0.2291 - val_loss: 0.2122
Epoch 755/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7906 - disc_loss: 0.1794 - rmse: 0.2368 - val_loss: 1.0004
Epoch 807/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7823 - disc_loss: 0.1822 - rmse: 0.2445 - val_loss: 1.0426
Epoch 808/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 20.2752 - disc_loss: 0.1833 - rmse: 0.3041 - val_loss: 0.2146
MissData :  save/  miss :  (9088, 12)
1/1 [==============================] - 0s 4ms/step - loss: 0.2243
MissData :  save/  miss :  (9088, 12)


In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [16]:
print(real_df_all.shape)
rnn_target_column = 'chl-a'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(51144, 36)
chl-a


In [17]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (35800, 36) val_df.shape :  (10229, 36) test_df.shape: (5115, 36)


In [18]:
#fake_df.shape[0]/8760

In [19]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [22]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  7
out_num_features :  1


In [23]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [24]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists('save/' + watershed):
        os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/geum/models/chlorophyll-a/


In [25]:
rnn_epochs

20

## 모델 학습

In [ ]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=50,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/50
10/10 [==============================] - ETA: 0s - loss: 0.1125 - mean_absolute_error: 0.2846 - nse: -3.8447
Epoch 00001: val_loss improved from inf to 0.11714, saving model to save/best_model.h5
10/10 [==============================] - 8s 800ms/step - loss: 0.1125 - mean_absolute_error: 0.2846 - nse: -3.8447 - val_loss: 0.1171 - val_mean_absolute_error: 0.3100 - val_nse: -1.4491
Epoch 2/50
10/10 [==============================] - ETA: 0s - loss: 0.0402 - mean_absolute_error: 0.1610 - nse: -0.6025
Epoch 00002: val_loss improved from 0.11714 to 0.05113, saving model to save/best_model.h5
10/10 [==============================] - 8s 772ms/step - loss: 0.0402 - mean_absolute_error: 0.1610 - nse: -0.6025 - val_loss: 0.0511 - val_mean_absolute_error: 0.1778 - val_nse: -0.0580
Epoch 3/50
10/10 [==============================] - ETA: 0s - loss: 0.0285 - mean_absolute_error: 0.1299 - nse: -0.0745
Epoch 00003: val_loss did not improve from 0.05113
10/10 [==============================

Epoch 23/50
10/10 [==============================] - ETA: 0s - loss: 0.0140 - mean_absolute_error: 0.0836 - nse: 0.4492
Epoch 00023: val_loss did not improve from 0.03463
10/10 [==============================] - 8s 772ms/step - loss: 0.0140 - mean_absolute_error: 0.0836 - nse: 0.4492 - val_loss: 0.0388 - val_mean_absolute_error: 0.1622 - val_nse: 0.1945
Epoch 24/50
10/10 [==============================] - ETA: 0s - loss: 0.0135 - mean_absolute_error: 0.0809 - nse: 0.4649
Epoch 00024: val_loss did not improve from 0.03463
10/10 [==============================] - 8s 762ms/step - loss: 0.0135 - mean_absolute_error: 0.0809 - nse: 0.4649 - val_loss: 0.0400 - val_mean_absolute_error: 0.1680 - val_nse: 0.1669
Epoch 25/50
10/10 [==============================] - ETA: 0s - loss: 0.0138 - mean_absolute_error: 0.0824 - nse: 0.4809
Epoch 00025: val_loss did not improve from 0.03463
10/10 [==============================] - 8s 765ms/step - loss: 0.0138 - mean_absolute_error: 0.0824 - nse: 0.4809 - v

## core / window.py / 

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
#train_df.shape, val_df.shape, test_df.shape

In [ ]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvbnfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************            
#                   SSSSSSSS               ******            ******              *****    *****                      
#        SSSS         S  SSS               ******            ******              *****    *****                     
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
